> Sorry about my English. 

> This is my first public competition.

#### let's start analyzing the solution. 
I took as a basis [LightGBM Baseline](https://www.kaggle.com/chocozzz/lightgbm-baseline)
When adding new features, I faced the problem of **kernel memory and time limit**. Upload the output data of other kernals into the kernal helped me to bypass this limitation. 
<img src="https://i.ibb.co/xhmgQ9q/2019-02-15-20-43-05.png" border="0" width="300px">
<br />
<img src="https://i.ibb.co/XF3Wz0c/2019-02-15-21-09-52.png" border="0"  width="500px">

I know that the game modes *solo, quad or duo* are very different in dynamics and tactics of the game.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_palette('bone')

train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

train.groupby('matchId')['matchType'].first().value_counts().plot.bar(ax=ax[0])

'''
solo  <-- solo,solo-fpp,normal-solo,normal-solo-fpp
duo   <-- duo,duo-fpp,normal-duo,normal-duo-fpp,crashfpp,crashtpp
squad <-- squad,squad-fpp,normal-squad,normal-squad-fpp,flarefpp,flaretpp
'''
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'
train['matchType'] = train['matchType'].apply(mapper)
train.groupby('matchId')['matchType'].first().value_counts().plot.bar(ax=ax[1])

So I just cut the data by game type (squad, duo, other) and **trained individual models/kernels for each mode** (kernel example [*pubg-lgb-part-squad*](https://www.kaggle.com/itslek/pubg-lgb-part-squad). ) Then simply "connected" results in other kernel - [*pubg-lgb-ensamble-lvl-1*](https://www.kaggle.com/itslek/pubg-lgb-ensamble-lvl-1).  
Thus, I was able to expand the number of features to 303 and achieve better convergence in a limited time. 

Only this decision has already given me **0.01974** [*pubg-lgb-ensamble-lvl-1*](https://www.kaggle.com/itslek/pubg-lgb-ensamble-lvl-1)

<img src="https://habrastorage.org/files/040/6ca/59e/0406ca59e7c243e1bffae413d1d40947.png" border="0" width="500px">

Next I used [Simple NN Baseline 4](https://www.kaggle.com/anycode/simple-nn-baseline-4)  and likewise **made a decision based NN** - [*pubg-nn-ensamble-lvl-1*](https://www.kaggle.com/itslek/pubg-nn-ensamble-lvl-1) 

its result was lower than the solution on  [*LightGBM*](https://www.kaggle.com/itslek/pubg-lgb-ensamble-lvl-1). (0.02052 vs 0.01974) but ** when combining solutions I got the total score increase to 0.01946.**

*Thanks to the organizers for the competition, it was interesting to practice with kernels*

### in Russian language

За основу я взял [LightGBM Baseline](https://www.kaggle.com/chocozzz/lightgbm-baseline)

Но При добавлении новых фитчей я столкнулся с проблемой ограничения кернелов по памяти и по времени. Обойти это ограничение мне помогла возможность подгружать в кернел результат работы других кернелов. 
Я знаю что игры в режимах solo, squad или duo сильно отличаються по динамике и по тактике игры. По этому я просто разбил данные по типу игры (squad, duo, other) и **обучал отдельные модели под каждый режи**м (пример кернала - [pubg-lgb-part-squad](https://www.kaggle.com/itslek/pubg-lgb-part-squad). ) Потом просто "склеивал" результаты в другом кернале - [pubg-lgb-ensamble-lvl-1](https://www.kaggle.com/itslek/pubg-lgb-ensamble-lvl-1).  
Тем самым я смог расширить количество фитчей до 303 и добиться лучшей сходимости в ограниченое время. Только это решение уже позволяло добиться **0.0197.**

Далее я звзял [Simple NN Baseline 4](https://www.kaggle.com/anycode/simple-nn-baseline-4) и аналогичным образом ** подготовил решение на основе NN ** [pubg-nn-ensamble-lvl-1](https://www.kaggle.com/itslek/pubg-nn-ensamble-lvl-1) его скор был ниже чем у решения на lgbm (0.02052 vs 0.0197) но **при объединении решений я получил общий прирост score до 0.01946.**

Вот и все)

In [ ]:
import os
import numpy as np
import pandas as pd

print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.
df_sub = pd.read_csv("../input/lightgbm-baseline/submission_adjusted.csv")
df_sub2 = pd.read_csv("../input/pubg-lgb-ensamble-lvl-1/submission_v8.csv")
df_sub3 = pd.read_csv("../input/pubg-nn-ensamble-lvl-1/submission_nn_ensamble_v5.csv")
df_test = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")

# STACK
df_sub["winPlacePerc"] = (df_sub2["winPlacePerc"] + df_sub3["winPlacePerc"]) / 2

df_sub = df_sub[["Id", "winPlacePerc"]]
df_test = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")


# Restore some columns
df_sub = df_sub.merge(df_test[["Id", "matchId", "groupId", "maxPlace", "numGroups"]], on="Id", how="left")

# Sort, rank, and assign adjusted ratio
df_sub_group = df_sub.groupby(["matchId", "groupId"]).first().reset_index()
df_sub_group["rank"] = df_sub_group.groupby(["matchId"])["winPlacePerc"].rank()
df_sub_group = df_sub_group.merge(
    df_sub_group.groupby("matchId")["rank"].max().to_frame("max_rank").reset_index(), 
    on="matchId", how="left")
df_sub_group["adjusted_perc"] = (df_sub_group["rank"] - 1) / (df_sub_group["numGroups"] - 1)

df_sub = df_sub.merge(df_sub_group[["adjusted_perc", "matchId", "groupId"]], on=["matchId", "groupId"], how="left")
df_sub["winPlacePerc"] = df_sub["adjusted_perc"]

# Deal with edge cases
df_sub.loc[df_sub.maxPlace == 0, "winPlacePerc"] = 0
df_sub.loc[df_sub.maxPlace == 1, "winPlacePerc"] = 1

# Align with maxPlace
# Credit: https://www.kaggle.com/anycode/simple-nn-baseline-4
subset = df_sub.loc[df_sub.maxPlace > 1]
gap = 1.0 / (subset.maxPlace.values - 1)
new_perc = np.around(subset.winPlacePerc.values / gap) * gap
df_sub.loc[df_sub.maxPlace > 1, "winPlacePerc"] = new_perc

# Edge case
df_sub.loc[(df_sub.maxPlace > 1) & (df_sub.numGroups == 1), "winPlacePerc"] = 0
assert df_sub["winPlacePerc"].isnull().sum() == 0

df_sub["winPlacePerc"] = df_sub["winPlacePerc"]


df_sub[["Id", "winPlacePerc"]].to_csv("submission_ensemble_lvl2_v4.csv", index=False)